# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [68]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [69]:
view_sentence_range = (0, 15)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 15:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician 

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [70]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    #I want a dictionary for words to id (vocab_to_int)
    #'' then I want a dictionary from id to word (int_to_vocab)
    
    #I'll just start off with a counter for the words
    counter = Counter(text)
    '''
    From Documentation: (For reference)
    A simple ascending sort is very easy: just call the sorted() function. 
    It returns a new sorted list:
    >>>
    >>> sorted([5, 2, 3, 1, 4])
    [1, 2, 3, 4, 5]
    
    Both list.sort() and sorted() accept a reverse parameter with a boolean 
    value. This is used to flag descending sorts.
    
    The method get() returns a value for the given key. If key 
    is not available then returns default value None.
    
    The dict() constructor builds dictionaries directly from sequences of key-value pairs:

    >>>
    >>> dict([('sape', 4139), ('guido', 4127), ('jack', 4098)])
    {'sape': 4139, 'jack': 4098, 'guido': 4127}
    
    The value of the key parameter should be a function that takes a single argument and
    returns a key to use for sorting purposes. This technique is fast because the key 
    function is called exactly once for each input record.
    '''
    
    #or .get()?
    vocab = sorted(counter, key = counter.get, reverse=True) #Here the sorted
                                                               #command makes life easier
    
    vocab_to_int = {word: ids for ids, word in enumerate(vocab)}#Make a dictionary for the words and id
    int_to_vocab = {ids: word for ids, word in enumerate(vocab)}
    
    
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [71]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {
            '.': '||Period||',
            ',': '||Comma||',
            '"': '||Quotation_Mark||',
            ';': '||Semicolon||',
            '!': '||Exclamation_Mark||',
            '?': '||Question_Mark||',
            '(': '||Left_Parentheses||',
            ')': '||Right_Parentheses||',
            '--': '||Dash||',
            '\n': '||Return||'
            }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [72]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [74]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [75]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    
    Input = tf.placeholder(tf.int32, shape=[None, None], name="input")
    
    Targets = tf.placeholder(tf.int32, shape=[None, None], name="targets")
    
    LearningRate = tf.placeholder(tf.float32, name="learning_rate")
    
    
    return (Input, Targets, LearningRate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [76]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    layers = 3 #The layers for the RNN cell
    
    #tf.contrib.rnn.BasicLSTMCell
    #inputs: 2-D tensor with shape [batch_size x input_size].
    LSTM_layer = tf.contrib.rnn.BasicLSTMCell(rnn_size) #tensorflow making life easy
    
    #tf.contrib.rnn.MultiRNNCell †Args:
    #inputs: 2-D tensor with shape [batch_size x input_size].
    Cell = tf.contrib.rnn.MultiRNNCell([LSTM_layer]*layers)
    
    
    #zero_state:
    # zero_state(
    #     batch_size,
    #     dtype
    # )
    InitialState = Cell.zero_state(batch_size, tf.float32)
    
    #     tf.identity
    # identity(
    #     input,
    #     name=None
    # )
    InitialState = tf.identity(InitialState, name='initial_state')
    
    
    
    return (Cell, InitialState)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [77]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
#     truncated_normal(
#     shape,
#     mean=0.0,
#     stddev=1.0,
#     dtype=tf.float32,
#     seed=None,
#     name=None
# )
    
    #got props for using truncated_normal in project 2
    embedded_input = tf.Variable(tf.truncated_normal((vocab_size, embed_dim), stddev=.01)) 
    
#     embedding_lookup(
#     params,
#     ids,
#     partition_strategy='mod',
#     name=None,
#     validate_indices=True,
#     max_norm=None
# )
# Looks up ids in a list of embedding tensors.

    #My parameter for this use would be embedded_input
    #input_data would be ids: A Tensor with type int32 or int64 containing the ids to be looked up in params.
    embedded_input = tf.nn.embedding_lookup(embedded_input, input_data)
    
    return embedded_input


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [78]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
#     dynamic_rnn(
#     cell,
#     inputs,
#     sequence_length=None,
#     initial_state=None,
#     dtype=None,
#     parallel_iterations=None,
#     swap_memory=False,
#     time_major=False,
#     scope=None
# )

# # 'state' is a tensor of shape [batch_size, cell_state_size]
# outputs, state = tf.nn.dynamic_rnn(rnn_cell, input_data,
#                                    initial_state=initial_state,
#                                    dtype=tf.float32)
    
    Outputs, Final_State = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    Final_State = tf.identity(Final_State, name = "final_state") #Because udacity so nicely asked
    
    return (Outputs, Final_State)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [79]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    embedding = get_embed(input_data, vocab_size, embed_dim)#Defined earlier
    Outputs, Final_State = build_rnn(cell, embedding)#cell, inputs (my embedding)
    
    #Here contrib makes my life easier
    
    #Can't recall if I already posted the documentation for this, but whatever
    
#     fully_connected(
#     inputs,
#     num_outputs,
#     activation_fn=tf.nn.relu,
#     normalizer_fn=None,
#     normalizer_params=None,
#     weights_initializer=initializers.xavier_initializer(),
#     weights_regularizer=None,
#     biases_initializer=tf.zeros_initializer(),
#     biases_regularizer=None,
#     reuse=None,
#     variables_collections=None,
#     outputs_collections=None,
#     trainable=True,
#     scope=None
# )

    Logits = tf.contrib.layers.fully_connected(Outputs, vocab_size, activation_fn=None)
    
    #This should be it
    
    return (Logits, Final_State)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [80]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    #The batches should be a Numpy array with the shape (number of batches, 2, batch size, sequence length)

    #The number of characters per batch would be batch_size*seq_length
    #The number of batches I'd want would be just be the floor div of the length
    #of the input text and characters per batch
    
    num_batches = len(int_text)//(batch_size*seq_length)
    
    batch_array = np.empty((num_batches, 2, batch_size, seq_length), dtype=int)
    
    #I can fill up my batches in the demonstrated manner using for loops
    #batch_array => There are 2 elements hard coded, so in the following for loops
    #I will just fill in the elements
    
    char_in_batch = batch_size*seq_length
    
    #Notice that the last target value in the last batch is the first input value 
    #of the first batch. In this case, 1. This is a common technique used when 
    #creating sequence batches, although it is rather unintuitive.
    
    #Because of that above rule, the inputs and targets will be looped around slightly
    #different int_text. I have 5000 as the size of my int_text, and 4480 as the size 
    #of int_text that makes it divisible by the character per batch. (640*7)
    
    int_text_end = num_batches*char_in_batch
    
    inputs = int_text[:int_text_end]
    targets = int_text[1:int_text_end] + [int_text[0]] #2nd elem needs to be list or else I get yelled at
    
    curr_batch = 0
    curr_sequence = 0
    
    for i in range(0, len(inputs), char_in_batch):#Goes by step of 640
        
        inputs_temp = inputs[i : char_in_batch+i]
        targets_temp = targets[i : char_in_batch+i]
        
        for j in range(0, len(inputs_temp), seq_length):#Goes by step of seq_length
            
            batch_array[curr_batch][0][curr_sequence] = inputs_temp[j : seq_length+j]
            batch_array[curr_batch][1][curr_sequence] = targets_temp[j: seq_length+j]
    
            #The following increments my batch or sequence depending on if I finished
            #filling up the current batch my for loop is working on
        
            if curr_batch < num_batches-1:
                curr_batch += 1
            else:
                curr_batch = 0
                curr_sequence += 1
                
        
    return batch_array    
    
    
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [81]:
#(64, 128 and 256)
#my get_batches func doesn't seem to be very efficient
#240 < num_epochs < 250 returns the best results, past
#that, my model starts to have tons of training loss

# Number of Epochs
num_epochs = 240
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 25
# Learning Rate
learning_rate = .01
# Show stats for every n number of batches
show_every_n_batches = 25

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [82]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [83]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.822
Epoch   1 Batch    4/21   train_loss = 6.384
Epoch   2 Batch    8/21   train_loss = 6.229
Epoch   3 Batch   12/21   train_loss = 6.221
Epoch   4 Batch   16/21   train_loss = 6.257
Epoch   5 Batch   20/21   train_loss = 6.246
Epoch   7 Batch    3/21   train_loss = 6.183
Epoch   8 Batch    7/21   train_loss = 6.170
Epoch   9 Batch   11/21   train_loss = 6.226
Epoch  10 Batch   15/21   train_loss = 6.159
Epoch  11 Batch   19/21   train_loss = 6.178
Epoch  13 Batch    2/21   train_loss = 6.184
Epoch  14 Batch    6/21   train_loss = 6.177
Epoch  15 Batch   10/21   train_loss = 6.127
Epoch  16 Batch   14/21   train_loss = 6.130
Epoch  17 Batch   18/21   train_loss = 6.081
Epoch  19 Batch    1/21   train_loss = 6.252
Epoch  20 Batch    5/21   train_loss = 6.091
Epoch  21 Batch    9/21   train_loss = 6.257
Epoch  22 Batch   13/21   train_loss = 6.159
Epoch  23 Batch   17/21   train_loss = 6.127
Epoch  25 Batch    0/21   train_loss = 6.123
Epoch  26 

Epoch 217 Batch   18/21   train_loss = 2.062
Epoch 219 Batch    1/21   train_loss = 1.795
Epoch 220 Batch    5/21   train_loss = 1.767
Epoch 221 Batch    9/21   train_loss = 1.582
Epoch 222 Batch   13/21   train_loss = 1.455
Epoch 223 Batch   17/21   train_loss = 1.376
Epoch 225 Batch    0/21   train_loss = 1.359
Epoch 226 Batch    4/21   train_loss = 1.235
Epoch 227 Batch    8/21   train_loss = 1.143
Epoch 228 Batch   12/21   train_loss = 1.040
Epoch 229 Batch   16/21   train_loss = 0.949
Epoch 230 Batch   20/21   train_loss = 0.927
Epoch 232 Batch    3/21   train_loss = 0.837
Epoch 233 Batch    7/21   train_loss = 0.836
Epoch 234 Batch   11/21   train_loss = 0.694
Epoch 235 Batch   15/21   train_loss = 0.649
Epoch 236 Batch   19/21   train_loss = 0.622
Epoch 238 Batch    2/21   train_loss = 0.643
Epoch 239 Batch    6/21   train_loss = 0.550
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [84]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [85]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [86]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    
    #     get_tensor_by_name
    # get_tensor_by_name(name)
    # Returns the Tensor with the given name.

    # This method may be called concurrently from multiple threads.

    # Args:
    # name: The name of the Tensor to return.
    # Returns:
    # The Tensor with the given name

    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [87]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    #int_to_vocab(probabilities)
    #Out of the probabilities, I want the highest one because that would show what my
    #system is looking for
    
    max_prob = np.argmax(probabilities)
    
    return int_to_vocab[max_prob]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [88]:
gen_length = 500
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(uneasy) oh, i can't wait till you guys what those name is yourse... which... someone at is.(hangs out) ah, when i did...
homer_simpson:(realizing) hey, guys, you sure. what have you and this man? he's one voice.
barney_gumble:(gently) my first prank, and?
carl_carlson: ah, homer. i said you're been good his one ceremony.
barney_gumble:(snorts) you were gonna get the stupid!
carl_carlson:(to camera) i think it's not.
lenny_leonard: oh, homer, and you're not the smell of my eggs is likes...
homer_simpson: what if i'd have no my place? jeez...
homer_simpson:(on noises) is that be. i'm--
barney_gumble: oh, i don't know.
businessman_#1:(into phone) hey, homer, i want to see you? anything?
homer_simpson:(to self) hello. what?
homer_simpson:(to moe) i got a bag of the shape in the first.(place the homer) heh. is it get the car, i ain't own her in a guy.
bart_simpson: yeah, some jerk. you don't have to make your women.
carl_carlson:(pleading) wow, well, that's your super thing?
mo

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.